In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

In [ ]:
# 📂 입력 및 출력 폴더 경로 설정
input_folder = [입력폴더]
output_folder = [출력폴더]

# 출력 폴더 생성
os.makedirs(output_folder, exist_ok=True)
print(f"📂 Output folder created at: {output_folder}")

In [ ]:
# CLAHE 객체 생성 (L 채널 대비 강화)
clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(12, 12))

# 입력 폴더의 모든 이미지 처리
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        image_path = os.path.join(input_folder, filename)
        try:
            print(f"📂 Processing: {filename}")

            # 🌈 1. Pillow로 이미지 불러오기 (RGB 모드)
            pil_image = Image.open(image_path).convert('RGB')
            image_np = np.array(pil_image)

            # 🟠 2. LAB 색공간 변환 및 L 채널 대비 강화
            lab = cv2.cvtColor(image_np, cv2.COLOR_RGB2LAB)
            l, a, b = cv2.split(lab)
            l_clahe = clahe.apply(l)  # CLAHE로 명암 대비 강화된 L 채널

            # 🟢 3. HSB 색공간 변환 및 H 채널 추출 (염증 강조)
            hsb = cv2.cvtColor(image_np, cv2.COLOR_RGB2HSV)
            h, s, v = cv2.split(hsb)

            # 🟡 4. L 채널과 H 채널 결합
            combined = cv2.merge((l_clahe, h, np.zeros_like(h)))  # L + H 결합
            combined_bgr = cv2.cvtColor(combined, cv2.COLOR_LAB2BGR)

            # 💾 5. 결과 저장 (Pillow 사용)
            output_path = os.path.join(output_folder, f"{filename}_LAB_HSB_Combined.png")
            Image.fromarray(combined_bgr).save(output_path, format='PNG')
            print(f"✅ Combined image saved at: {output_path}")

            # 🖼️ 6. 시각화 (비교)
            plt.figure(figsize=(12, 6))
            plt.subplot(2, 3, 1); plt.imshow(image_np); plt.title('Original Image'); plt.axis('off')
            plt.subplot(2, 3, 2); plt.imshow(l, cmap='gray'); plt.title('LAB - L (Lightness)'); plt.axis('off')
            plt.subplot(2, 3, 3); plt.imshow(l_clahe, cmap='gray'); plt.title('LAB - L (CLAHE)'); plt.axis('off')
            plt.subplot(2, 3, 4); plt.imshow(h, cmap='gray'); plt.title('HSB - H (Hue)'); plt.axis('off')
            plt.subplot(2, 3, 5); plt.imshow(combined); plt.title('Combined (L+H)'); plt.axis('off')
            plt.subplot(2, 3, 6); plt.imshow(combined_bgr); plt.title('Combined BGR'); plt.axis('off')

            plt.suptitle(f'LAB-HSB Combined Analysis: {filename}', fontsize=16)
            plt.show()

        except Exception as e:
            print(f"❌ Error processing {filename}: {e}")

print("🎉 모든 이미지의 LAB-HSB 결합 분석 및 저장이 완료되었습니다.")